In [43]:
#IMPORT PACKAGES
from rdflib import Graph
import rdflib
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d

import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from scipy.linalg import orthogonal_procrustes
import matplotlib.pyplot as plt
import geomapi.tools.progresstools as pt

from sklearn.cluster import DBSCAN
from PIL import Image

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t8_utils as t8

from typing import Dict, Any, Tuple,List


In [89]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[0]

print(path)
input_folder_t4=path/'data'/'t4'/'train' 
input_folder_t6=path/'data'/'t6'/'train'
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t8'/ 'train'
os.makedirs(output_folder, exist_ok=True)

#parameters
grid_resolution = 0.01

/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024


Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import Graph

In [6]:
graphfiles=utl.get_list_of_files(input_folder_t6,'.ttl')
for f in graphfiles[:1]: #only read the first one
    print(f'processing {ut.get_filename(f)}...')      
    wallNodes=tl.graph_path_to_nodes(f)
    
    for n in wallNodes:
        n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(n.orientedBounds)))

    print(f'{len(wallNodes)} wallNodes detected!')

processing 05_MedOffice_01_F2_walls...
145 wallNodes detected!


Import PCD

In [7]:
pcdfiles=utl.get_list_of_files(input_folder_t4,'.laz')
for f_pcd in pcdfiles[:1]: #only read the first one
    pcdNodes = t8.match_graph_with_las(f_pcd,class_dict, nodes = wallNodes, getResources=True, getNormals=False)

processing 05_MedOffice_01_F2_small1...


In [8]:
for n in wallNodes: 
    n.derivedFrom = next((p for p in pcdNodes if n.object_id == p.object_id), None)

Convert dtrings from the graph to arrays

In [9]:
for n in wallNodes:
    n.startpoint = ut.literal_to_array(n.start_pt) #np.asarray(n.start_pt[1:-1].split(), dtype=float)
    n.endpoint = ut.literal_to_array(n.end_pt) #np.asarray(n.end_pt[1:-1].split(), dtype=float)
    n.normal = ut.literal_to_array(n.normal) #np.asarray(n.normal[1:-1].split(), dtype=float)
    n.height = float(n.height)
    n.name = n.subject.split('///')[-1]
    if n.width == 0.127 or n.width == 0.2:
        n.singleFaced = True
    else:
        n.singleFaced = False
    

Convert the full point cloud into a mesh and add it to a raycasting scene

In [10]:

#Cut a part out of the full resolution pointcloud
# joined_pcd = full_res_point_cloud_o3d.crop(expanded_bounding_box)
#Create a messh from this point cloud 
octree=pt.pcd_to_octree(gmu.join_geometries([n.pcd for n in pcdNodes]),12) #if octree is None else octree
mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh


#Create a identity array containing the color so this can be retrieved afterwards
original_colors=np.asarray(mesh.vertex_colors)
indices=np.asarray(mesh.triangles)[:,0]
triangle_colors=original_colors[indices]
#append black color at the end of the array for the invalid hits
triangle_colors=np.vstack((triangle_colors,np.array([0,0,0])))

# Create raycasting scene
scene = o3d.t.geometry.RaycastingScene()
mesh=o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene.add_triangles(mesh) 

# Calculate the size of each octree node based on octree depth and overall size
def calculate_node_size(octree_depth, octree_size):
    num_voxels_per_dim = 2 ** octree_depth
    voxel_size = octree_size / num_voxels_per_dim
    return voxel_size

# Example usage:
octree_depth = octree.max_depth  # Example value for max_depth
octree_size = octree.size  # Example size of the octree in world units
voxel_size = calculate_node_size(octree_depth, octree_size)
print("Voxelsize = ", voxel_size)
octree = None

Voxelsize =  0.018880786132812497


Using the pointcloud and wall data to create an ortho foto of the wall and use object detection
(Can also be used to retrieve other elements)

In [11]:
image_resolution = 0.01

for n in wallNodes:
    
    length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
    surface = length * n.height
    image_size = (int(length / image_resolution), int(n.height / image_resolution))
    n.orthos = []
    

    if not surface < 3 and n.height > 1.5 and length > 0.8:  
        #Create an ortho of the dominant side of the wall      
        ortho = t8.create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = n.normal, scene=scene, triangle_colors = triangle_colors)
        ortho = t8.fill_black_pixels(ortho, region = 10)
        n.orthos.append(ortho)
        #Also create an ortho of the other side of the wall
        if not n.singleFaced: #Single faced wall only needs one side
            ortho = t8.create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = -n.normal, scene=scene, triangle_colors = triangle_colors, dominant = False)
            ortho = t8.fill_black_pixels(ortho, region = 10)
            n.orthos.append(ortho)

In [12]:
import torch
from PIL import Image
from torchvision.ops import box_convert

# Grounding DINO
from groundingdino.models import build_model
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.utils import clean_state_dict
from groundingdino.util.inference import annotate, load_image, predict

import supervision as sv

# segment anything
# from segment_anything import build_sam, SamPredictor 
import numpy as np


# diffusers
import torch

from huggingface_hub import hf_hub_download

In [13]:
# Use this command for evaluate the Grounding DINO model
# Or you can download the model by yourself
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"

In [14]:
from torch.cuda.amp import autocast
with autocast():
    groundingdino_model = t8.load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device="cuda:1")

final text_encoder_type: bert-base-uncased
Model loaded from /home/sdegeyter/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [62]:
count = 0
pointcloud = []


for n in wallNodes:
    TEXT_PROMPT = "Door"
    BOX_TRESHOLD = 0.20
    TEXT_TRESHOLD = 0.5
    
    n.boxes = []
    n.logits = []
    n.phrases = []
    
    if len(n.orthos) > 0:
        for ortho in n.orthos:
            boxes = None
            image = load_image(Image.fromarray((ortho * 255).astype(np.uint8)))

            boxes, logits, phrases = predict(
                model=groundingdino_model, 
                image=image, 
                caption=TEXT_PROMPT, 
                box_threshold=BOX_TRESHOLD, 
                text_threshold=TEXT_TRESHOLD
            )
            
            n.boxes.append(boxes)
            n.logits.append(logits)
            n.phrases.append(phrases)                  

In [517]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [518]:
def indices_of_values_smaller_than_max(arr):
    """
    Return the indices of all values in the array that are smaller than the maximum value.

    Args:
        arr (np.ndarray): The input array.

    Returns:
        np.ndarray: Indices of values smaller than the maximum value.
    """
    if np.asarray(arr).size == 0:
        return np.array([])  # Return an empty array if the input is empty
    
    max_value = np.max(arr)
    indices = np.where(arr < max_value)[0]
    return indices

In [543]:
from utils.t8_utils import calculate_percentage_black_pixels


doorNodes = []
pointcloud = []

wall = 0
door_count = 0
image = 0
id_count = 4000
    
door_count = 0
doorNodes = []

class_id = 5
for n in wallNodes:#[72:73]: #^22:24 interessante muren
    potential_door_boxes_wall = []
    potential_door_info_wall = []
    
    if (len(n.orthos) == 1 and len(n.boxes[0]) > 0) or len(n.orthos) == 2 and (len(n.boxes[0]) > 0 or len(n.boxes[1]) > 0) :
        for j, boxes in enumerate(n.boxes):
            potential_door_boxes_face = []
            potential_door_info_face = []
            if len(boxes) > 0: 
                for i, box in enumerate(boxes):
                    probability = float(n.logits[j][i])
                    opening_width = round(int(np.asarray(box)[2]*n.orthos[j].shape[1])* image_resolution, 2)    
                    opening_height = round(int(np.asarray(box)[3]*n.orthos[j].shape[0]) * image_resolution, 2)
                    
                    detection_center_u = int(np.asarray(box)[0]*n.orthos[j].shape[1]) * image_resolution
                    detection_center_v = int(np.asarray(box)[1]*n.orthos[j].shape[0]) * image_resolution
                    
                    reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                    # print((str(j) + "-" +str(image)))
                    score = t8.is_door(probability, opening_width, opening_height, reference_level, prob_weight=0.15, width_weight=0.15, height_weight=0.3, ref_level_weight=0.3, name = (str(j) + "-" +str(image)) )
                    
                    if score >= 0.6:
                        # annotated_frame = annotate(image_source= n.orthos[j], boxes=box.unsqueeze(0), logits=torch.from_numpy(np.array([score])), phrases=[n.phrases[j][i]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Wall_" + str(wall) + "-Face_" + str(j) +"-" +str(image) +"-"+  str(score) +'-DOOR.png')))
                        # image += 1
                        
                        box1 = copy.deepcopy(box)
                        box = box.unsqueeze(0)

                        if j == 1:
                            box1[0] = 1-box1[0]

                        potential_door_boxes_face.append(np.asarray(box1))
                        potential_door_info_face.append([opening_width, opening_height, image, reference_level, score])
                            
            potential_door_boxes_wall.append(potential_door_boxes_face)
            potential_door_info_wall.append(potential_door_info_face)
            
        #merge largely overlapping bounding boxes
        for i, face_boxes in enumerate(potential_door_boxes_wall):
            face_boxes = t8.find_and_merge_high_iou_boxes(face_boxes, threshold=0.8, info = potential_door_info_wall[i])
    
        #For Double faced walls look if the detection is found on both sides.
        if len(potential_door_boxes_wall) == 2 and not len(potential_door_boxes_wall[0]) == 0 and not len(potential_door_boxes_wall[1]) == 0:
            # print("Double Faced")
            matches, unmatched_boxes0, unmatched_boxes1 = t8.find_best_matches(potential_door_boxes_wall[0], potential_door_boxes_wall[1], potential_door_info_wall[0], potential_door_info_wall[1], iou_weight=0.33, param_weight=0.33, doorness_weight=0.33)

            for id0, id1, bestscore in matches:
                if not id0 == None and not id1 == None:
                    # compare the parameters of the different matches
                    info0 = potential_door_info_wall[0][id0]
                    info1 = potential_door_info_wall[1][id1]
          
                    # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(np.asarray([potential_door_boxes_wall[0][id0]])), logits=torch.from_numpy(np.array([info0[-1]])), phrases=[n.phrases[0][id0]])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Wall_" + str(wall) + "-Face_0-" +str(image) +"-"+  str(info0[-1]) +'-DOOR.png')))
                    # image += 1
                    
                    # annotated_frame = annotate(image_source= n.orthos[1], boxes=torch.from_numpy(np.asarray([potential_door_boxes_wall[1][id1]])), logits=torch.from_numpy(np.array([info1[-1]])), phrases=[n.phrases[1][id1]])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Wall_" + str(wall) + "-Face_1-" +str(image) +"-"+  str(info1[-1]) +'-DOOR.png')))
                    # image += 1
                    
                    detectionbox = t8.combine_boxes(potential_door_boxes_wall[0][id0], potential_door_boxes_wall[1][id1])
                    
                    opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[0], image_resolution = image_resolution)
                    # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %((str(count)), bestscore, reference_level, opening_width, opening_height))
                    
                    boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                    
                    doornode = BIMNode(
                        name= "Doors_" + str(door_count),
                        axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                        startpoint= boundaryPoints[0],
                        endpoint= boundaryPoints[1],
                        doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                        height = opening_height,
                        doornessScore = bestscore,
                        singleFaced = False,
                        depth = n.width,
                        object_id = class_id *4000 + door_count,
                        blackness = percentage_black_pixles,
                        host = n)
                    
                    doorNodes.append(doornode)
                    print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                    # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                    door_count += 1 
                    
            #If the detection is not found on both sides of the wall but it is a double faced wall we add a penalty
            for id0 in unmatched_boxes0: #Two faced walls with a detection on only one side
                score = potential_door_info_wall[0][id0][4]-0.2
                if score > 0.7:
                    detectionbox = np.array([potential_door_boxes_wall[0][id0]])
                    
                    opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[0], image_resolution = image_resolution)
                    
                    boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
        
                    doornode = BIMNode(
                        name= "Doors_" + str(door_count),
                        axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                        startpoint= boundaryPoints[0],
                        endpoint= boundaryPoints[1],
                        doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                        height = opening_height,
                        doornessScore = score,
                        singleFaced = True,
                        depth = n.width,
                        object_id = class_id *4000 + door_count,
                        blackness = percentage_black_pixles,
                        host = n)
                    
                    doorNodes.append(doornode)
                    print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                    # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                    door_count += 1 
                
            for id1 in unmatched_boxes1: #Two faced walls with a detection on only one side
                score = potential_door_info_wall[1][id1][4]-0.2
                if score > 0.7:
                    detectionbox = np.array([potential_door_boxes_wall[1][id1]])

                    opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[1], image_resolution = image_resolution)
                      
                    boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                        
                    doornode = BIMNode(
                        name= "Doors_" + str(door_count),
                        axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                        startpoint= boundaryPoints[0],
                        endpoint= boundaryPoints[1],
                        doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                        height = opening_height,
                        doornessScore = score,
                        singleFaced = True,
                        depth = n.width,
                        object_id = class_id *4000 + door_count,
                        blackness = percentage_black_pixles,
                        host = n)
                    
                    doorNodes.append(doornode)
                    print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                    # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                    door_count += 1 

        else: #For single faced walls
            for i, box in enumerate(potential_door_boxes_wall[0]):
                #This are single faced walls so the penalty is less.
                score = potential_door_info_wall[0][i][4]-0.1
                # image_resource = t8.extract_box_with_margin(n.orthos[0], np.array([box])[0])
                # image_resource = image_resource[...,::-1] # BGR to RGB
                # Image.fromarray(image_resource).save(os.path.join(output_folder,("BlackDoor_" + str(door_count) + '.png')))
                # bl_px = t8.calculate_percentage_black_pixels(image_resource)
                
                if score > 0.75: #and bl_px > 0.6:
                    detectionbox = np.array([box])
                    
                    opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[0], image_resolution = image_resolution)
                     
                    boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                        
                    doornode = BIMNode(
                        name= "Doors_" + str(door_count),
                        axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                        startpoint= boundaryPoints[0],
                        endpoint= boundaryPoints[1],
                        doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                        height = opening_height,
                        doornessScore = score,
                        singleFaced = True,
                        depth = n.width,
                        object_id = class_id *4000 + door_count,
                        blackness = percentage_black_pixles,
                        host = n)
                    
                    doorNodes.append(doornode)
                    print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                    # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                    door_count += 1 
          
            if len(potential_door_boxes_wall) > 1:    
                for i, box in enumerate(potential_door_boxes_wall[1]):
                    #This are single faced walls so the penalty is less.
                    score = potential_door_info_wall[1][i][4]-0.1
                    
                    
                    if score > 0.75:
                        detectionbox = np.array([box])
                        
                        opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[1], image_resolution = image_resolution)
                            
                        boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                        
                        
                        doornode = BIMNode(
                            name= "Doors_" + str(door_count),
                            axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                            startpoint= boundaryPoints[0],
                            endpoint= boundaryPoints[1],
                            doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                            height = opening_height,
                            doornessScore = score,
                            singleFaced = True,
                            depth = n.width,
                            object_id = class_id *4000 + door_count,
                            blackness = percentage_black_pixles,
                            host = n)
                        
                        doorNodes.append(doornode)
                        print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                        # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                        door_count += 1 
        
    wall += 1
    
if len(doorNodes) >= 1:
    for n in doorNodes:
        pointList=[]
        points=np.asarray(n.axis.points)
        # pointList.extend(points+n.sign*n.normal*n.width/2)
        pointList.extend(points+n.host.normal*n.host.width/2)

        # pointList.extend(points-n.sign*n.normal*n.width/2)
        pointList.extend(points-n.host.normal*n.host.width/2)

        pointList.extend(np.array(pointList)+np.array([0,0,n.height]))
        pcd=o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pointList))

        box=pcd.get_oriented_bounding_box()
        n.resource = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(box)
        n.resource.paint_uniform_color(ut.literal_to_array(n.host.color))
        n.doorBox=o3d.geometry.LineSet.create_from_oriented_bounding_box(box)
        n.doorBox.paint_uniform_color([0,0,1])
        
        n.center = t8.compute_center(n.startpoint, n.endpoint)
        n.cartesianTransform = copy.deepcopy(n.host.cartesianTransform)
        n.cartesianTransform[:3,3] = n.center
        n.normal = n.host.normal
        n.rotation = t8.get_angle_with_x_axis(n.startpoint, n.endpoint)
        
    lines = []
    # Function to calculate the distance between two points
    def calculate_distance(point1, point2):
        return np.linalg.norm(np.array(point1) - np.array(point2))


    avg_doorWidth = np.mean(np.array([e.doorWidth for e in doorNodes]))
    print(avg_doorWidth)
    #remove single faced doors that are in a 90 degree angle with double faced doors.
    dbscan = DBSCAN(eps=1.5*avg_doorWidth, min_samples=1, metric=t8.calculate_distance) #eps=1.5*avg_doorWidth
    labels = dbscan.fit_predict([n.center for n in doorNodes])

    cluster_ids = [-1] * len(doorNodes)
    unique_labels = set(labels)

    current_cluster_id = 0
    for label in unique_labels:
        cluster_lines = [doorNodes[i] for i in range(len(doorNodes)) if labels[i] == label]
        for line in cluster_lines:
            line_index = None
            for i, l in enumerate(doorNodes):
                if np.array_equal(l, line):
                    line_index = i
                    break
            cluster_ids[line_index] = current_cluster_id
        current_cluster_id += 1

    all_clusters_indices = []

    # Get unique cluster labels
    unique_labels = np.unique(cluster_ids)

    # Iterate through each unique cluster label
    for cluster_label in unique_labels:
        if not cluster_label == -1:
            cluster_indices = [i for i, label in enumerate(cluster_ids) if label == cluster_label]
            all_clusters_indices.append(cluster_indices)
        else: 
            for i, label in enumerate(cluster_ids):
                if label == cluster_label:
                    all_clusters_indices.append([i])
    print(all_clusters_indices) 
    # FP = []
    for cluster_indices in all_clusters_indices:
        if len(cluster_indices) > 1:
            doors_to_check = []
            for i in cluster_indices:
                doors_to_check.append([doorNodes[i].name, doorNodes[i].doornessScore, doorNodes[i].blackness])
                # if doorNodes[i].singleFaced and not doorNodes[i].blackness <0.1:
                #     # print(f"door: {doorNodes[i].name} will be removed")
                #     FP.append(doorNodes[i].subject)
            print(doors_to_check)

    
        # ids = indices_of_values_smaller_than_max(blackness)
        # for doornode_id in ids:
            

    # doorNodes = [doornode for doornode in doorNodes if doornode.subject not in FP]

    door_count = len(doorNodes)
    # print(f'name: {n.name}, score: {n.doornessScore} ,doorWidth: {n.doorWidth}, height: {n.height}')
    
print(f"Number of doors: {door_count}")

Doors_0 : Door Score: 0.7873185962438585 => Referencelevel: 0.01; Width: 0.85; height: 2.2
Doors_1 : Door Score: 0.7782544881105424 => Referencelevel: 0.01; Width: 0.63; height: 2.19
Doors_2 : Door Score: 0.9826827863515967 => Referencelevel: 0.0; Width: 1.3; height: 2.31
Doors_3 : Door Score: 0.850829178094864 => Referencelevel: 0.02; Width: 1.16; height: 2.21
Doors_4 : Door Score: 0.8595124125480653 => Referencelevel: 0.01; Width: 0.88; height: 2.1
Doors_5 : Door Score: 0.9618811101730369 => Referencelevel: 0.02; Width: 0.92; height: 2.09
Doors_6 : Door Score: 0.8628721674283347 => Referencelevel: 0.03; Width: 0.89; height: 2.07
Doors_7 : Door Score: 0.9774538766160762 => Referencelevel: 0.02; Width: 1.05; height: 2.2
Doors_8 : Door Score: 0.8504207770029705 => Referencelevel: 0.01; Width: 0.94; height: 2.12
Doors_9 : Door Score: 0.7751466025908789 => Referencelevel: 0.05; Width: 1.26; height: 2.2
Doors_10 : Door Score: 0.8329984784126283 => Referencelevel: 0.04; Width: 1.16; height:

## Ground truth

In [539]:
# groundtruthPath = '/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t0/05_MedOffice_01_F2_doors.json'
# groundtruthNodes = []

# with open(groundtruthPath) as gt:
#     data = json.load(gt)
    
# for item in data:
#     doorNode = Node(
#         id = item['id'],
#         width = item['width'],
#         height = item['height'],
#         depth = item['depth'],
#         center = item['loc'], 
#         rotation = item['rotation'], 
#         host = item['host_id']
#     )
#     groundtruthNodes.append(doorNode)
# print(len(groundtruthNodes))


In [540]:
# #match the graphs with the training data

# input_folder_gt=path/'data'/'t1'/'train'

# gt_files=utl.get_list_of_files(input_folder_gt,'.ttl')
# gt_files_obj=utl.get_list_of_files(input_folder_gt,'.obj')
# for f_gt,f_gt_obj in zip(gt_files[:1],gt_files_obj[:1]): #only read the first one
    
#     #import objects
#     mesh_dict=utl.load_obj_and_create_meshes(f_gt_obj)
    
#     #import graph
#     pcdNodes_gt=tl.graph_path_to_nodes(graphPath=str(f_gt))
#     pcdNodes_gt_doors=[n for n in pcdNodes_gt if n.class_id==4]

#     #add objects to the nodes
#     for n in pcdNodes_gt_doors:
#         mesh=next((mesh for name, mesh in mesh_dict.items() if name == n.name),None)
#         n.resource=mesh
#         n.lineset=o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
#         n.lineset.paint_uniform_color(ut.literal_to_array(n.color))
#         #compute the normal from start to end point
#         n.resource.compute_triangle_normals()
        
#         n.normal =np.asarray( n.resource.triangle_normals)[11]
       
        
#         # direction = (ut.literal_to_array(n_gt.end_pt) - ut.literal_to_array(n.start_pt))/np.linalg.norm(ut.literal_to_array(n_gt.end_pt) - ut.literal_to_array(n.start_pt))
#         # n.normal = np.cross(direction, np.array([0, 0, 1]))
#         # if np.linalg.norm(normal) == 0:
#         #     normal = np.cross(direction, np.array([0, 1, 0]))
#         # n.normal = normal / np.linalg.norm(normal)
        
        
#     print(f'{len(pcdNodes_gt_doors)} pcdNodes_gt_doors detected!') # there are only 144 walls in the training data, yet 161 are found here
#     print(f'{len(doorNodes)} doorNodes detected!') # there are only 144 walls in the training data, yet 161 are found here
    
#     #match the walls with the ground truth
#     for n in doorNodes:
        
#         #create lineset
#         n.lineset=o3d.geometry.LineSet.create_from_triangle_mesh(n.resource)
#         # n.lineset.paint_uniform_color(ut.literal_to_array(n.color))
                
#         #find the corresponding ground truth wall 
#         pose=n.cartesianTransform[:3,3]
#         distances=[np.linalg.norm(pose-n.cartesianTransform[:3,3]) for n in pcdNodes_gt_doors]
#         idx=np.argmin(distances)
#         n_gt=pcdNodes_gt_doors[idx]
#         # n_gt.color=n.color
#         n.corresponding_gt=n_gt.name
#         n.corresponding_normal=n_gt.normal
        
#         #create a line from the n.resource.get_center() along the n.normal in red and along the n_gt.normal in green
#         line=o3d.geometry.LineSet()
#         line.points=o3d.utility.Vector3dVector([n.resource.get_center(),n.resource.get_center()+n.normal])
#         line.lines=o3d.utility.Vector2iVector([[0,1]])
#         line.colors=o3d.utility.Vector3dVector([[1,0,0]])
#         n.lineset+=line
#         line=o3d.geometry.LineSet()
#         line.points=o3d.utility.Vector3dVector([n.resource.get_center(),n.resource.get_center()+n_gt.normal])
#         line.lines=o3d.utility.Vector2iVector([[0,1]])
#         line.colors=o3d.utility.Vector3dVector([[0,1,0]])
#         n.lineset+=line
        
        
#         #make comparison
#         # n.center_diff=np.round(np.min(np.array([np.linalg.norm(ut.literal_to_array(n_gt.loc)-n.center)])),2)
#         n.height_diff=np.round(np.abs(n_gt.height-n.height),2)
#         n.width_diff=np.round(n_gt.width-n.doorWidth,2)
#         print(f'door {n.get_name()}, height_diff of {n.height_diff} (gt height is {np.round(n_gt.height,2)}), width_diff of {n.width_diff} (gt width is {np.round(n_gt.width,2)})')
#     print(f'len of elements width_def >0.03: {len([e for e in doorNodes if np.abs(e.width_diff)>0.03])}/{len(doorNodes)}')
#     print(f'len of elements height_diff>0.1: {len([e for e in doorNodes if np.abs(e.height_diff)>0.1])}/{len(doorNodes)}')
#     print(f'mean height: {np.mean(np.array([e.height for e in doorNodes]))}')
#     print(f'mean width: {np.mean(np.array([e.doorWidth for e in doorNodes]))}')

## EXPORT

In [541]:

t8.write_obj_with_submeshes(os.path.join(output_folder,f'{ut.get_filename(f)}_doors.obj') , [n.resource for n in doorNodes], [n.name for n in doorNodes])

In [542]:
reform_name='_'.join(ut.get_filename(f).split('_')[:4])+'_doors'

json_data=t8.doors_to_json(doorNodes)
with open(os.path.join(output_folder,reform_name+'.json'), 'w') as file:
    json.dump(json_data, file, indent=4)
print("JSON data written to file:", os.path.join(output_folder,reform_name+'.json'))

JSON data written to file: /home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t8/train/05_MedOffice_01_F2_doors.json
